In [1]:
import piton
import piton.datasets
import pickle
import numpy as np

from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM
from piton.featurizers import save_to_file, load_from_file
from piton.featurizers.featurizers import TextFeaturizer, _get_text_embeddings
from typing import Tuple, List
import multiprocessing
import datetime
import os

/home/rthapa84/.conda/envs/env_piton/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_to_file(object_to_save, path_to_file: str):
    """Save object to Pickle file."""
    os.makedirs(os.path.dirname(path_to_file), exist_ok=True)
    with open(path_to_file, "wb") as fd:
        pickle.dump(object_to_save, fd)

def load_from_file(path_to_file: str):
    """Load object from Pickle file."""
    with open(path_to_file, "rb") as fd:
        result = pickle.load(fd)
    return result

In [3]:
# Please update this path with your extract of piton as noted in previous notebook. 
# PATH_TO_PITON_DB= '/share/pi/nigam/data/som-rit-phi-starr-prod.starr_omop_cdm5_deid_2022_09_05_extract2'
PATH_TO_SAVE_MATRIX = "/local-scratch/nigam/projects/rthapa84/data"
TEXT_EMBEDDINGS_PATH = "test_diabetes_text_embeddings.pickle"

path_to_model = "/local-scratch/nigam/projects/clmbr_text_assets/models/Clinical-Longformer"
# path_to_model = "/local-scratch/nigam/projects/clmbr_text_assets/models/Bio_ClinicalBERT"
path_to_labeled_patients = "/local-scratch/nigam/projects/rthapa84/data/HighHbA1c_labeled_patients_v3.pickle"
database_path = "/local-scratch/nigam/projects/ethanid/som-rit-phi-starr-prod.starr_omop_cdm5_deid_2022_09_05_extract2"
path_to_save = "/local-scratch/nigam/projects/rthapa84/data/"
num_threads = 20
num_threads_gpu = 2
min_char = 100
max_char = 10000
max_length = 1024
padding = True
truncation = True
chunk_size = 10
num_patients = None

In [4]:
tokenized_text_list = load_from_file(os.path.join(path_to_save, "temp_tokenized_text.pickle"))
print(len(tokenized_text_list))

5


In [12]:
model = AutoModel.from_pretrained(path_to_model)

Some weights of the model checkpoint at /local-scratch/nigam/projects/clmbr_text_assets/models/Clinical-Longformer were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at /local-scratch/nigam/projects/clmbr_text_assets/models/Clinical-Longformer and are newly initialized: ['longformer.poole

In [15]:
from tqdm import tqdm
embeddings = []
for tokenized_data in tqdm(tokenized_text_list):
    outputs = model(**tokenized_data)
    batch_embedding_tensor = outputs.last_hidden_state[:, 0, :].squeeze()
    batch_embedding_numpy = batch_embedding_tensor.cpu().detach().numpy()
    embeddings.append(batch_embedding_numpy)

embeddings = np.concatenate(embeddings)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:31<00:00, 30.27s/it]


In [16]:
embeddings.shape

(50, 768)